# 12장 폼 패턴들

장고의 폼은 강력한 동시에 유연하며 자유자재로 확장 가능한 구조로 되어 있다.

그 덕분에 장고 폼은 장고 어드민과 클래스 기반 뷰에서 광범위하게 이용되고 있으며 대부분의 장고 API 프레임워크에서 모델폼 또는 그와 비슷한 종류의 구현이 유효성 검사를 위해 쓰이고 있다.

이번 장에서는 장고에서 최고의 기능이라 불리는 폼, 모델, 클래스 기반 뷰를 잘 이용하는 방법을 다루겠다.

## 12.1 패턴 1: 간단한 모델폼과 기본 유효성 검사기

가장 간단한 데이터 변경 폼은 ModelForm인데, 이는 기본 유효성 검사기를 있는 그대로 수정 없이 이용하는 것이다.

예제 12.1

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_1.py

* Flavor 모델을 FlavorCreateView와 FlavorUpdateView에서 이용하도록 한다.
* 두 뷰에서 Flavor 모델에 기반을 둔 ModelForm을 자동 생성한다.
* 생성된 ModelForm이 Flavor 모델의 기본 필드 유효성 검사기를 이용하게 된다.

장고에서는 기본으로 데이터 유효성 검사기를 제공한다. 하지만 기본 유효성 검사만으로는 충분하다고 볼 수 없다. 

따라서 다음 패턴에서 커스텀 필드 유효성 검사기를 제작하는 방법을 알아보겠다.

## 12.2 패턴 2: 모델폼에서 커스텀 폼 필드 유효성 검사기 이용하기

프로젝트의 모든 앱의 타이틀 필드가 'Tasty'로 시작되어야 한다면 어떻게 할까?

예제 12.2

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_2.py

validate_tasty()를 다른 종류의 디저트 모델에 적용하기 위해 우선 TastyTitleAbstractModel 이라는 프로젝트 전반에서 이용할 수 있는 추상화 모델을 추가한다.

예제 12.3

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_3.py

마지막 두 줄이 TastyTitleAbstractModel 을 추상화 모델로 만들어준다.

예제 12.4

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_4.py

타이틀 말고 다른 필드에 적용하고 싶을 때 - (예제 12.2를 이용)

예제 12.5

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_5.py

유효성 검사기를 사용하는 방법의 장점은 validate_tasty()의 코드를 변경하지 않고 바로 이용할 수 있다.

새로운 곳에 import 하는 것으로 바로 이용할 수 있다. 

커스텀 폼을 뷰에 추가.

장고의 모델 기반 수정 뷰(model-based edit view)는 뷰의 모델 속성(attribute)을 기반으로 모델폼을 자동으로 생성해 준다.

이 기본 생성된 모델폼에 우리의 커스텀 FlavorForm을 오버라이딩하겠다.

예제 12.6

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_6.py

이제 FlavorCreateView와 FlavorUpdateView 뷰에서는 입력된 데이터의 유효성 검사를 위해 FlavorForm을 이용하게 되었다.

## 12.3 패턴 3: 유효성 검사의 클린 상태 오버라이딩하기

* 다중 필드에 대한 유효성 검사
* 이미 유효성 검사가 끝난 데이터베이스의 데이터가 포함된 유효성 검사

두 가지 경우 전부 커스텀 로직으로 clean() 또는 clean_<field_name>() 메서드를 오버라이딩할 수 있는 최적의 경우이다.

1) clean() 메서는 어떤 특별한 필드에 대한 정의도 가지고 있지 않기 때문에 두 개 또는 그 이상의 필드들에 대해 서로 간의 유효성을 검사하는 공간이 된다.

2) 클린(clean) 유효성 검사 상태는 영속 데이터(presistent data)에 대해 유효성을 검사하기에 좋은 장소이다. 이미 유효성 검사를 일부 마친 데이터에 대해 불필요한 데이터베이스 연동을 줄일 수 있다.

예) 재고가 없는 아이스크림을 선택하는 경우를 피하기 위해 clean_slug() 메서드를 추가하자

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_7.py

예) 고객이 쵸콜릿을 너무 많이 주문했을 때, 에러 내기

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_8.py

## 12.4 패턴 4: 폼 필드 해킹하기(두 개의 CBV, 두 개의 폼, 한 개의 모델)

하나의 모델에 두 개의 뷰와 폼이 엮여 있는 경우

이를 위해 장고 폼을 수정하여 특별한 작동(custom behavior)을 하는 특수한 폼을 생성

예제 12.9

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_9.py

나쁜 예제 12.1

폼으로 필드의 정의를 복사, 붙이기 하는 대신에 간단하게 ModelForm의 __init__() 메서드에서 새로운 속성을 적용하면 된다.

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_10.py

장고의 폼도 결국 파이썬 클래스이고, 장고의 폼 또한 객체로 실체회되고 슈퍼클래스가 되어 다른 클래스를 상속하기도 한다.

따라서 아이스크림 스토어 폼에서 상속을 이용함으로써 많은 코드를 줄일 수 있다.

예제 12.11

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_11.py

모든 폼 클래스를 구현했으니 이제 구현된 폼 클래스들을 IceCreamStore의 create와 update 뷰에서 이용해 보자.

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_12.py

## 12.5 패턴 5: 재사용 가능한 검색 믹스인 뷰

각기 다른 두 개의 모델에 연동되는 두 개의 뷰에 하나의 폼을 재사용할 수 있는지 알아보자.

두 개의 모델 전부 title이라는 필드가 있다고 가정하자.

이 예에서는 하나의 클래스 기반 뷰로 Flavor와 IceCreamStore 모델 두 개에 대해 간단한 검색 기능을 구현해 보겠다.

뷰에 간단한 검색 믹스인을 만들어보자.

예제 12.13

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_13.py

flavor 뷰

예제 12.14

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_14.py

아이스크림 스토어 뷰

예제 12.15

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_15.py

폼의 경우 다음과 같이 각각의 ListView의 HTML 안에 포함시키면 된다.

예제 12.16

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_16.html

예제 12.17

https://github.com/twoscoops/two-scoops-of-django-1.8/blob/master/code/chapter_12_example_17.html

이제 두 개의 뷰에 같은 믹스인을 이용하게 되었다. 믹스인은 코드를 재사용하는 좋은 방법이지만 단일 클래스에서 너무 많은 믹스인을 이용하면 코드 유지보수가 매우 어려워진다.

## 12.6 요약

ModelForm과 클래스 기반 뷰, 기본 유효성 검사기를 이용하는 가장 단순한 형태의 패턴부터 시작하여 커스텀 유효성 검사기까지 적용해 보았다.

그리고 더 복잡한 유효성 검사를 살펴보았다. 클린 메서드들을 오버라이딩하는 부분도 살펴보았으며 두 개의 뷰와 두 개의 뷰에 해당하는 폼을 하나의 모델에 연동하는 방법도 알아보았다.

마지막으로 각기 다른 두 개의 앱에 같은 폼을 이용하기 위해 재사용이 가능한 믹스인을 생성하는 경우를 다루어 보았다.